### Sand Rate, Mixed Velocity and Erosion calculation from Pysand

__Go through the following steps :__

1. Update TAG, time range, sampling frequency in sec
2. Update R, GF and d_p input
3. Update well input data
4. Update segment name to get correct polynomials for Z.values
5. Update the well name to load correct back ground noise input
6. Update the class variables (D, rho, MW, mu) for the correct well input
7. Select vendor name for the step values
8. Select erosion model and material
<br> Erosion models available : bend, tee, straight_pipe, welded_joint, manifold, reducer, probes, flexible
<br> Materials available : carbon_steel, duplex, ss316, inconel, grp_epoxy, aluminium, dc_05_tungsten, etc

__How the Erosion models selected, and the required input__ :
- bend(v_m, rho_m, mu_m, R, GF, D, d_p, material='duplex', rho_p=2650)
- tee(v_m, rho_m, mu_m, GF, D, d_p, material='duplex', rho_p=2650):
- straight_pipe(v_m, D)
- welded_joint(v_m, rho_m, D, d_p, h, alpha=60, location='downstream', material='duplex')
- manifold(v_m, rho_m, mu_m, GF, D, d_p, Dm, material='duplex')
- reducer(v_m, rho_m, D1, D2, d_p, GF=2, alpha=60, material='duplex')
- probes(v_m, rho_m, D, d_p, alpha=60, material='duplex')
- flexible(v_m, rho_m, mu_m, mbr, D, d_p, material='duplex')
- choke_gallery(v_m, rho_m, mu_m, GF, D, d_p, R_c, gap, H, material='cr_37_tungsten')


For first time user :
1. Install Tagreader :
https://wiki.equinor.com/wiki/index.php/Tagreader-python

2. Install Pysand :
https://github.com/equinor/pysand/blob/master/README.md

***
MMEG

            Row Labels  15/9-C-2 AHT2
86 2021-07-30 12:00:00           36.0
0.9599505923327949


 2022-06-24 01:32:15,198 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,201 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,205 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,209 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,213 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,217 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,221 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,224 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,227 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,231 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:15,236 WARNING: The model has got negative value(s) 

 2022-06-24 01:32:16,152 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,153 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,153 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,154 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,154 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,155 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,155 WARNING: Mix density, rho_m, is outside RP-O501 model boundaries (1-1500 kg/m3).
 2022-06-24 01:32:16,156 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,156 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,158 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,159 WARNING: The model has got negat

 2022-06-24 01:32:16,228 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,230 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,230 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,231 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,231 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,232 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,233 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,233 WARNING: Mix density, rho_m, is outside RP-O501 model boundaries (1-1500 kg/m3).
 2022-06-24 01:32:16,234 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,235 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,237 WARNING: The model has got negat

 2022-06-24 01:32:16,306 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,307 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,308 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,309 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,309 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,310 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,310 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,310 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,311 WARNING: Mix density, rho_m, is outside RP-O501 model boundaries (1-1500 kg/m3).
 2022-06-24 01:32:16,312 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,312 WARNING: The model has got negat

 2022-06-24 01:32:16,501 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,503 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,504 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,505 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,505 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,506 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,506 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,507 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,507 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,508 WARNING: Mix density, rho_m, is outside RP-O501 model boundaries (1-1500 kg/m3).
 2022-06-24 01:32:16,508 WARNING: The model has got negat

 2022-06-24 01:32:16,597 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,598 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,600 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,601 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,603 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,604 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,605 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,606 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,606 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,607 WARNING: The model has got negative value(s) of T and returned nan.
 2022-06-24 01:32:16,607 WARNING: Mix density, rho_m, is outside RP-O5

c:\appl\python3_8\lib\site-packages\plotly\tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



                          time        WHP          Raw        WHT  Wing  \
0    2022-06-14 00:00:00+02:00  14.268757  4346.159180  46.823589   1.0   
1    2022-06-14 00:01:00+02:00  14.282372  2794.674072  46.906967   1.0   
2    2022-06-14 00:02:00+02:00  14.295985  5199.583008  46.786869   1.0   
3    2022-06-14 00:03:00+02:00  14.309600  4352.266113  46.849678   1.0   
4    2022-06-14 00:04:00+02:00  14.323214  4053.839844  46.744572   1.0   
...                        ...        ...          ...        ...   ...   
8635 2022-06-19 23:55:00+02:00  13.893104  4366.890625  32.254063   1.0   
8636 2022-06-19 23:56:00+02:00  14.006404  3684.654785  32.337509   1.0   
8637 2022-06-19 23:57:00+02:00  14.067230  3854.633789  31.425903   1.0   
8638 2022-06-19 23:58:00+02:00  13.875858  2242.450195  31.547899   1.0   
8639 2022-06-19 23:59:00+02:00  13.900229  4560.506348  30.963629   1.0   

        DSC       DSCT  E_rate (mm/yr)  E (mm/tonn)  Qs (g/s)  Vmix (m/s)  \
0     9.725  14.078125

In [2]:
import pysand as ps
import numpy as np
import sys
import tagreader
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from datetime import datetime, timedelta
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly import tools


#Read data from PI. 
c=tagreader.IMSClient("SLP", "PI")
c.cache = None
c.connect()

#1. Update TAG, time range, sampling frequency in sec
#end = str(datetime.now())
#start = str(datetime.now()- timedelta(days=7))


start = datetime(2022,6,14,0,0,0)
end = datetime(2022,6,19,23,59,0)
#end = start+ timedelta(0,36000)

sampling=60
df1 = c.read(['18-PT___157_','18-AT___836.Rawdata','18-TT___151_','18-XA___821_','18-PT___832_','18-TT___831_'], 
             str(start), str(end), sampling)
df1.rename(columns={'18-PT___157_' : 'WHP','18-AT___836.Rawdata':'Raw','18-TT___151_': 'WHT',
                    '18-XA___821_':'Wing','18-PT___832_':'DSC','18-TT___831_':'DSCT'}, inplace=True)
# time data as column and not only index
df1=df1.reset_index()
n=len(df1['WHP'])


#2. Update R, GF and d_p input

R=1.5 #bend radius, how many times ID's
GF = 3 # geometry factor
d_p = 0.3 # sand size 250 um
deg1=40 # well inclination


##############
#2. Performance curve, write correct well name
df3=pd.read_excel('G:\\UPN\\T-S_Sleipner\\PETEK_B&B_RESU\\Protek\\Megawati\\PERF_CURVES.xlsx', 
                  sheet_name='Sheet2',engine='openpyxl')
filter=df3['WELL_NAME'].isin(['C-2'])
df3=df3.loc[filter]


latest=df3['VALID_FROM_DATE'].max()
df3=df3.loc[(df3['VALID_FROM_DATE'] >= latest)]
#df3=df3.loc[(df3['VALID_FROM_DATE'] == '02/01/2022')]

df3=df3.sort_values(["X_VALUE","Y_VALUE"],ascending=True)

WGR=df3['WGR'].max()
CGR=df3['CGR'].max()
Pres_pc= df3['X_VALUE']
Gas_pc= df3['Y_VALUE']


############



#3. Update well input data , sheet name in P
#df3 : perfor curve, df2: Polynomials for Z values, df4:PVT tables, df5:densities, df6: polynomials for background noise
#Only Update the well name in df3 to get correct perf curve input
dfP=pd.read_excel('G:\\UPN\\T-S_Sleipner\\PETEK_B&B_RESU\\Protek\\Megawati\\PRESSURES.xlsx', 
                  sheet_name='datum',engine='openpyxl',skiprows=3)
dfZ=pd.read_excel('G:\\UPN\\T-S_Sleipner\\PETEK_B&B_RESU\\Protek\\Megawati\\PRESSURES.xlsx', 
                  sheet_name='SLØ_HUGIN',engine='openpyxl')


df4=pd.read_excel('G:\\UPN\\T-S_Sleipner\\PETEK_B&B_RESU\\Protek\\Megawati\\PVT.xlsx', 
                  sheet_name='PVT',index_col=0,engine='openpyxl')
df5=pd.read_excel('G:\\UPN\\T-S_Sleipner\\PETEK_B&B_RESU\\Protek\\Megawati\\PVT.xlsx', 
                  sheet_name='rho',index_col=0,engine='openpyxl')
df6=pd.read_excel('G:\\UPN\\T-S_Sleipner\\PETEK_B&B_RESU\\Protek\\Megawati\\PVT.xlsx', 
                  sheet_name='ZERO_SAND',index_col=0,engine='openpyxl')

MW=df5.loc['SO']['MW_gas']
rho_w=df4.loc['S-3']['Water dens (kg/m3)']
rho_o=df4.loc['S-3']['Oil dens (kg/m3)']
mu_o=df4.loc['S-3']['Oil visc (kg/ms)']
mu_w=df4.loc['S-3']['Water visc (kg/ms)']
mu_g=df4.loc['S-3']['Gas Visc (kg/ms)']


# 4. Use correct well name 
dfP=dfP[['Row Labels','16/7-A-3 AH']]
dfP=dfP[dfP['16/7-A-3 AH'].notna()]

latest=dfP['Row Labels'].max()
dfP=dfP.loc[dfP['Row Labels'] >= latest]

Z=(np.interp(dfP['16/7-A-3 AH'],dfZ['bar'],dfZ['Z'])).mean()

print(dfP)
print(Z)




#5. Update the well name to load correct polynomials for the background noise.
#Polynomials are taken from field watch
zero_pol=df6.loc['C-02a'][2:]


#print(CGR,WGR,zero_pol,pol)

### Define Class for the various calculations. 
#6. Update the class variables (D, rho, MW, mu) for the correct well input

class Calculate:
    D=0.1244 #pipe X-section ID
    
    
       
    def __init__(self, df_pres, df_gas, wgr, cgr, df_whp,df_wht, df_zeropol,df_DSC,df_wing,df_DSCT):
        self.df_pres = df_pres
        self.df_gas = df_gas
        self.wgr = wgr
        self.cgr = cgr
        self.df_whp=df_whp
        self.df_wht = df_wht              
        self.df_zeropol = df_zeropol
        self.df_DSC=df_DSC
        self.wing=df_wing
        self.df_DSCT=df_DSCT
        
       
    def qgas(self):
        Qg = np.interp(self.df_whp, self.df_pres, self.df_gas)
        return Qg if self.wing ==1 else 0
            
        
    def qwat(self):
        Qw = self.wgr*np.interp(self.df_whp, self.df_pres, self.df_gas)
        return Qw 
    
    def qoil(self):
        Qo = self.cgr*np.interp(self.df_whp, self.df_pres, self.df_gas)
        return Qo 
    
    def qglr(self):
        glr = self.qgas/(self.qwat+self.qoil)
        return glr  
              
    
    
    def v_m(self):
        v_m = ps.fluidproperties.mix_velocity(self.df_whp, self.df_wht, self.qoil(),self.qwat(), 
                                              self.qgas(),Z,Calculate.D)
        return v_m if v_m >0 else 0.01
  
    
    def zerofunc(self):
        poly_zero = np.poly1d(self.df_zeropol)
        zero= poly_zero(self.v_m())
        #zero=1500
        return zero if self.qgas()>100000 else 70000
        
        
    
    def rho_m(self):
        rho_m = ps.mix_density(self.df_whp, self.df_wht, self.qoil(),self.qwat(),self.qgas(), 
                              rho_o, rho_w, MW,Z)
        return rho_m if rho_m >0 else 0.01
    
    def mu_m(self):
        mu_m = ps.fluidproperties.mix_viscosity(self.df_whp, self.df_wht,self.qoil(),self.qwat() , 
                                               self.qgas(),mu_o, mu_w, mu_g,Z)
        return mu_m if mu_m >0 else 0.01
    
    def rho_l(self):
        rho_l=(rho_w*self.qwat()+rho_o*self.qoil())/(self.qwat()+self.qoil())
        return rho_l if rho_l >0 else 0.01
    
    def mu_l(self):
        mu_l=(mu_w*self.qwat()+mu_o*self.qoil())/(self.qwat()+self.qoil())
        return mu_l if mu_l >0 else 0.01
    
    def v_dsc(self):
        v_dsc = ps.fluidproperties.mix_velocity(self.df_DSC,self.df_DSCT,self.qoil(),self.qwat(),self.qgas(),Z,Calculate.D)
        return v_dsc if self.wing ==1 else 0
        

### Go through the time WHP data, raw data, and calculate for the erosion
vmix_dsc=[]
step=np.ones([n,1])
E=np.ones([n,1])
E_rate=np.ones([n,1])
Qs=np.ones([n,1])
qgas=[]
zero=[]    
vmix=[]
rho_mix=[]
crit_v=[]
stoke_v1=[]

for i in range(0,n):
    calc = Calculate(Pres_pc,Gas_pc,WGR, CGR,df1['WHP'][i], df1['WHT'][i], zero_pol,df1['DSC'][i],df1['Wing'][i],df1['DSCT'][i])            
    vmix.append(calc.v_m())
    vmix_dsc.append(calc.v_dsc())  
    zero.append(calc.zerofunc())
    qgas.append(calc.qgas())
    #critical velocity for moving dunes and dispersed flow
    crit_v.append(ps.transport.hydro(Calculate.D,calc.rho_l(),calc.mu_l(),d_p))
    
    #stokes velocity for the actual well inclination
    stoke_v1.append(ps.transport.stokes(calc.rho_m(),calc.mu_m(),d_p,deg1))
    
    #7. Update vendor std_step_clampon, or emerson
    step[i]=ps.asd.std_step_emerson(calc.v_m(),1/CGR)
    Qs[i]=ps.asd.sand_rate(df1['Raw'][i], calc.zerofunc(), step[i])
    rho_mix.append(calc.rho_m())
    #8. Select erosion model and material
    #E is relative erosion in mm/ton
    E[i]=ps.erosion.bend(calc.v_m(), calc.rho_m(), calc.mu_m(), R, GF, Calculate.D, d_p, material='carbon_steel')
    # E_rate is erosion rate in mm/yr
    E_rate[i] =ps.erosion.erosion_rate(E[i],Qs[i][0])

df1['E_rate (mm/yr)']=E_rate
df1['E (mm/tonn)']=E
df1['Qs (g/s)']=Qs
df1['Vmix (m/s)']=vmix
df1['Critical vel (m/s)']=crit_v
df1['Well Name'] = "C-2"
df1['Background Noise'] = zero
df1['Sand lift rate']=stoke_v1
df1['Gas rates (Sm3/d)']=qgas
df1['Mixed density']=rho_mix
df1['Vmix DSC (m/s)']=vmix_dsc

#Plot the results
rcParams['figure.figsize']=20,30
figure,axis=plt.subplots(5)    
#use.values in the date time to avoid pandas converter 
axis[0].plot(df1['time'].values,df1['E_rate (mm/yr)'].values,color='r')
axis[0].set_ylabel('erosion rate (mm/yr)')
axis[1].plot(vmix, label ='mixed velocity')
axis[1].plot(crit_v, label='critical velocity')
axis[1].plot(stoke_v1, label='max stokes v')
axis[1].set_ylabel('velocity (m/s)')
axis[1].legend(loc="upper right")
axis[1].set_ylabel('V (m/s)')
axis[2].plot(Qs)
axis[2].set_ylabel('Sand rate (g/s)')
axis[3].plot(df1['E (mm/tonn)'])
axis[3].set_ylabel('Erosion(mm/ton)')
axis[4].plot(df1['time'].values, qgas, color='r')
axis[4].set_ylabel('Gas rate')

#print(E)
#print(Qs)
#print(crit_v)
df1=df1.loc[:, ~df1.columns.isin(['DSCT'])]
plt.show()

df1['SR hourly_avg']=df1['Qs (g/s)'].rolling(window=60).mean()
df1['SR daily_avg']=df1['Qs (g/s)'].rolling(window=1440).mean()

##plotly
trace1=go.Scatter(x=df1['time'],y=zero,
                 mode='lines',name='Zero')

trace2=go.Scatter(x=df1['time'],y=df1['Raw'],
                 mode='lines+markers',name='Raw')


trace3=go.Scatter(x=df1['time'],y=df1['Qs (g/s)'].rolling(window=60).mean(),
                 mode='lines',name='Hourly AVG')

trace4=go.Scatter(x=df1['time'],y=df1['Qs (g/s)'].rolling(window=1440).mean(),
                 mode='lines',name='Daily AVG')

trace5=go.Scatter(x=df1['time'],y=df1['Qs (g/s)'],
                 mode='lines',name='Sand Rates')



fig=tools.make_subplots(rows=2,cols=1,subplot_titles=['ASD','Sand Rates '])
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,1)
fig.append_trace(trace3,2,1)
fig.append_trace(trace4,2,1)
fig.append_trace(trace5,2,1)

fig['layout'].update(title='C-2')

pyo.plot(fig)


#append results to csv file, mode a (append)
#df1.to_csv('G:\\UPN\\T-S_Sleipner\\PETEK_B&B_RESU\\Protek\\Megawati\\sand\\C2.csv', mode='a',index=False, header=None)
#df1.to_csv('G:\\UPN\\T-S_Sleipner\\PETEK_B&B_RESU\\Protek\\Megawati\\sand\\C2.csv', index=False, header=True)
print(df1)




                    